In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle
import cv2 as cv
import matplotlib.pyplot as plt
import praw
import requests
import json
import csv
import time
import datetime

In [6]:
# creating a reddit app instance to collect post data from r/india submreddit
reddit = praw.Reddit(client_id='u0_eiRibrOBr5w', client_secret='wYMSPOuqFfUGubC81Nnn7eCvIsE', user_agent='reddit-scrap-1', username='macabdul9', password='Sudo$0#1')

In [7]:
subm = reddit.submission(id="fxqifi")

In [8]:
subm.title

'RSS says Tablighi Jamaat conduct not reflection on all Muslims, they’re aiding govt in fight'

In [9]:
def get_comments(url, num_comments):
    # get top 20 comments if total comments are more than that
    subm = reddit.submission(url=url)

    # comments = subm.comments.list()
    # if len(comments) > 10:
    #     comments = comments[:10]

    comments_body = []
    sum_score = 0
    count = 0
    if num_comments > 0:
        for i, comment in enumerate(subm.comments.list()):

            # There exist some comments which has no body
            try : 
                comments_body.append(comment.body)
            except:
                comments_body.append('')

            # There exist some comments which has not given a score
            try:
                sum_score += comment.score
            except:
                sum_score += 0            
            count += 1
            # We only need 10 comments
            if (i+1)%10==0:
                break
    try:
        mean = sum_score/count
    except:
        mean = 0.0
    

    return " ".join(comments_body), mean

In [10]:
def getPushshiftSubData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=2000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [11]:
features = [
    'allow_live_comments', 'author', 'author_premium', 'can_mod_post', 'contest_mode', 'created_utc',
    'full_link', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_self', 'is_video',
    'link_flair_text', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18',
    'parent_whitelist_status', 'permalink', 'pinned', 'score', 'selftext', 'send_replies', 'spoiler',
    'stickied', 'title', 'total_awards_received', 'url'
]
custom_features = [
    'comments', 'mean_comment_score'
]

In [12]:
def collectSubData(subm):
    
    # extracting all information about a submission
    feature_values = [subm.get(key) for key in features]    
    comments, mean_comment_score = get_comments(subm.get('full_link'), subm.get('num_comments')) 
    feature_values.append(comments)
    feature_values.append(mean_comment_score)
    subStats[subm.get('id')] = feature_values

In [22]:
#Subreddit to query
sub='india'

#before and after dates
after = "1548979200"  # Fri, 01 Feb 2019 00:00:00 
before = "1551398399" # Thu, 28 Feb 2019 23:59:59  
query = "" # title should have either null string more 
subCount = 0
subStats = {}

In [28]:
data = getPushshiftSubData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    # print(len(data))
    if subCount % 1000 == 0:
        print(f'{subCount} submission collected')
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftSubData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1551255137&before=1551398399&subreddit=india
2019-02-28 23:56:50
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1551378410&before=1551398399&subreddit=india
2019-03-01 05:28:00
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1551398280&before=1551398399&subreddit=india
0


In [18]:
with open("Submissions-Feb-2020.pkl", "wb") as f:
    pickle.dump(subStats, f)

In [19]:
df = pd.DataFrame(data=list(subStats.values()), columns = features + custom_features, index=None)

In [20]:
df.to_csv("Submissions-Feb-2020.csv", index=False)